In [86]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [87]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [88]:
# extract text from given .pdf file
def pdf2text(pdf_path, text_path, page_start, page_end):
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(page_start, page_end + 1):
            # print(i)
            page = reader.pages[i]
            text = text + "".join(page.extract_text().splitlines())

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# ind = 1
# pdf_file_name = str(index) + ".pdf"
# pdf_folder_path = fpath.litera_pdf_folder
# pdf_path = os.path.join(pdf_folder_path, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# if not pdf2text(pdf_path, text_path, start_page, end_page):
#     print("Error: pdf2text() failed")
# ---------------------end of test code---------------------

In [89]:
def rename_pdf(newname, folder_of_download, time_to_wait=60):
    time_counter = 0
    filename = max([f for f in os.listdir(folder_of_download)], key=lambda xa :   os.path.getctime(os.path.join(folder_of_download,xa)))
    while '.part' in filename:
        time.sleep(1)
        time_counter += 1
        if time_counter > time_to_wait:
            raise Exception('Waited too long for file to download')
    filename = max([f for f in os.listdir(folder_of_download)], key=lambda xa :   os.path.getctime(os.path.join(folder_of_download,xa)))
    os.rename(os.path.join(folder_of_download, filename), os.path.join(folder_of_download, newname))

In [90]:
# download pdf to specified folder given pdf_url and ind
def download_pdf(pdf_url, pdf_folder_path):  
    url = pdf_url
    if url != url:
        raise Exception("pdf_url is np.nan")
    else:
        source = url.split("://")[1].split("/")[0]
    
    # pdf_path = os.path.join(pdf_folder_path, file_name)
      
    # get the pdf content
    # options = webdriver.ChromeOptions()
    # options.add_experimental_option('prefs', {
    # "download.default_directory": pdf_folder_path, #Change default directory for downloads
    # "download.prompt_for_download": False, #To auto download the file
    # "download.directory_upgrade": True,
    # "download.open_pdf_in_system_reader": False,
    # "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    # })
    # driver = webdriver.Chrome(options=options)
    # # response = driver.get(pdf_url)
    # driver.navigate().to(url)
    # time.sleep(5)

    # try:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         driver.close()
    #         return True
    # except:
    #     print(f'Failed downloading PDF:', url)
    #     driver.close()
    #     # print(f'HTTP response status code: {response.status_code}')
    #     return False

    # response = requests.get(pdf_url, headers=plib.headers)
    response = requests.get(url, stream=True, allow_redirects=True, headers=plib.headers)
    
    # download the .pdf file to the pdf_file_path folder
    # write content in pdf file
    pdf_path = os.path.join(pdf_folder_path, file_name)
    
    if response.status_code == 200:
        with open(pdf_path, 'wb') as pdf_object:
            pdf_object.write(response.content)
            return True
    else:
        print(f'Failed downloading PDF:' + 'pdf_url')
        print(f'HTTP response status code: {response.status_code}')
        return False
# --------------------start of test code--------------------
# pdf_url = 'https://www.sciencedirect.com/science/article/pii/S0896627320300052/pdfft?md5=3f0648c6385e6fae3a5a73b053903014&pid=1-s2.0-S0896627320300052-main.pdf'
pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"
pdf_url, code = plib.get_final_redirected_url(pdf_url)
print(pdf_url)
ind = 4
file_name = str(ind) + ".pdf"
pdf_folder_path = fpath.litera_pdf_folder
download_pdf(pdf_url, pdf_folder_path)
rename_pdf(file_name, pdf_folder_path, time_to_wait=60)
# ---------------------end of test code---------------------

https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219
Failed downloading PDF:pdf_url
HTTP response status code: 403


In [92]:
url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"
response = requests.get(url, stream=True, allow_redirects=True, headers=plib.headers)
response.status_code

403